In [1]:
import os
import json
import nltk
from nltk.corpus import stopwords
stopwerdz = []
count = 0
with open('stopwords.txt',mode = 'r') as stopwerds:
    while 1:
        werd = stopwerds.readline()
        werd = werd.strip('\n')
        stopwerdz.append(werd)
        count = count + 1
        if count > 433:
            break
        

In [2]:
with open('andy.txt',mode='r',encoding = 'utf-8') as f:
    count = 0
    rows = 0
    sentences = {}
    sentence = []
    while 1:
        word = f.readline()
        rows = rows + 1
        if word != '&#128512\n':
            word = word.replace('\n',' ')
            word = word.replace('\t','')
            sentence.append(word)
            if word=='endend':
                break
        else:
            sentences[count]=sentence
            sentence=[]
            count = count+1
            

In [ ]:
sentences

In [ ]:
stopwerdz

In [5]:
stw = stopwords.words('romanian')
stw.extend(stopwerdz)

In [8]:
filtered_sentences={}
for key in sentences.keys():  
    filtered_sentence=[]
    for w in sentences[key]:
        filtered_word=''
        for letter in w:
            if (letter.isalpha()):
                filtered_word+=letter
            else:
                if(letter == '~'):
                    filtered_word+=' '
        if filtered_word not in stw and filtered_word != '' and len(filtered_word) >= 2: 
            filtered_sentence.append(filtered_word) 
    filtered_sentences[key]=filtered_sentence

In [ ]:
filtered_sentences

In [11]:
def convert_to_string(werds):
    return ' '.join(werds)

In [17]:
for key in filtered_sentences.keys():
    filtered_sentences[key]=convert_to_string(filtered_sentences[key])

In [27]:
print(filtered_sentences[0])


începe programare vaccinare problemă din prima pentru că programare sine ordine alfabetic județ vâlcea Vaslui sine redenumit rapid Âlcea Aslui român reuși intra platformă reuși ieși programat vaccin popular vaccin pfizer urmat Internal server Error invalid page STS avertiza circula spam prinț nigerian moșteni milion doză vaccin turism înflori român city break dărăban decât Paris locuitor flămânzi ruga aduce mâncare veni vaccin colonel gheorghiță în sfârșit crescut grad colonel Gheorghe mesaj Iohannis drag român găsi loc programare însemna ocupat pesediști programa mediaș capcană mânare vaccin sine aloca doză întru județ exista sălaj


In [41]:
import csv
with open('filtered_output', 'w') as csv_file:  
    writer = csv.writer(csv_file,delimiter='\t')
    for key, value in filtered_sentences.items():
       writer.writerow([value])